In [2]:
import numpy as np
from scipy import stats
import os
import pandas as pd
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
import math
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import sqlite3
%matplotlib inline

In [4]:
data_remove_e = pd.read_csv("data_for_rohan.csv",low_memory=False)

In [12]:
data_remove_e = data_remove_e.drop('Unnamed: 0', axis=1)

In [14]:
data_remove_rpl = data_remove_e.drop(['COMPL_RPY_1YR_RT', 'NONCOM_RPY_1YR_RT', 'LO_INC_RPY_1YR_RT', 'MD_INC_RPY_1YR_RT'\
                                      , 'HI_INC_RPY_1YR_RT', 'DEP_RPY_1YR_RT', 'IND_RPY_1YR_RT', 'PELL_RPY_1YR_RT', \
                                      'NOPELL_RPY_1YR_RT', 'FEMALE_RPY_1YR_RT', 'MALE_RPY_1YR_RT', 'FIRSTGEN_RPY_1YR_RT', \
                                      'NOTFIRSTGEN_RPY_1YR_RT'], axis=1)

In [19]:
nan_counts = data_remove_rpl.count(axis=1)
proportion_nan = []
proportion_nan[:] = [1 - ((float) (a) / (float) (len(data_remove_rpl.columns))) for a in nan_counts]

In [30]:
df_p_nan = pd.DataFrame(proportion_nan)
delete_these_rows = df_p_nan.ix[df_p_nan[0] > 0.9].index.values
data_thinner = data_remove_rpl.drop(data_remove_rpl.index[delete_these_rows])
data_thinner.shape

(1729, 245)

In [52]:
data_thinner = data_thinner.drop(['count_ed', 'count_nwne_p6', 'count_wne_p6', 'md_earn_wne_p6', 'pct10_earn_wne_p6',\
                                     'pct25_earn_wne_p6', 'pct75_earn_wne_p6', 'pct90_earn_wne_p6', 'sd_earn_wne_p6', \
                                     'gt_25k_p6', 'mn_earn_wne_inc1_p6', 'mn_earn_wne_inc2_p6',\
                                     'mn_earn_wne_inc3_p6'], axis=1)

In [90]:
data_thinner = data_thinner.reset_index()
data_thinner = data_thinner.drop('index', axis=1)
keep_indices = np.isfinite(data_thinner.mn_earn_wne_p6)

In [91]:
drop_this = []
for i in range(len(keep_indices)):
    if keep_indices.ix[i] == False:
        drop_this.append(i)
data_super_thin = data_thinner.drop(data_thinner.index[drop_this])

In [92]:
data_super_nans = data_super_thin.replace(np.nan,'nan')

In [93]:
data_super_nans.to_csv('data_super_nan.csv', encoding='utf-8')

In [130]:
data_needs_headers = pd.read_csv("data_imputed.csv",low_memory=False,header=None)

In [126]:
data_super_nans = data_super_nans.drop('level_0', axis=1)

In [131]:
data_still_needs_headers = data_needs_headers.drop(0,axis=1)
data_still_needs_headers.columns = data_super_nans.columns

In [132]:
data_still_needs_headers.to_csv('data_full_final.csv', encoding='utf-8')

In [134]:
data_z = stats.zscore(data_still_needs_headers, axis=1)

In [136]:
data_z_full = pd.DataFrame(data=data_z,columns=data_super_nans.columns)
data_z_full.ix[:,'UNITID'] = data_still_needs_headers.ix[:,'UNITID']
data_z_full.to_csv('data_z_full_final.csv', encoding='utf-8')

In [141]:
y_full = data_still_needs_headers.ix[:,'mn_earn_wne_p6']
y_full_z = data_z_full.ix[:,'mn_earn_wne_p6']

In [142]:
x_full = data_still_needs_headers.drop('mn_earn_wne_p6', axis=1)
x_full_z = data_z_full.drop('mn_earn_wne_p6', axis=1)

In [146]:
y_full.to_csv('y_full.csv', encoding='utf-8',index=False)
y_full_z.to_csv('y_full_z.csv', encoding='utf-8',index=False)
x_full.to_csv('x_full.csv', encoding='utf-8',index=False)
x_full_z.to_csv('x_full_z.csv', encoding='utf-8',index=False)

In [155]:
data_all_names = pd.read_csv('old_data/data_full.csv', low_memory=False)

In [156]:
data_all_names = data_all_names.drop('Unnamed: 0', axis=1)

In [158]:
inst_names = []
for i in range(len(data_still_needs_headers)):
    find_id = data_still_needs_headers.ix[i,'UNITID']
    for j in range(len(data_all_names)):
        if data_all_names.ix[j,'UNITID']==find_id:
            inst_names.append(data_all_names.ix[j,'INSTNM'])

In [171]:
data_still_needs_headers.index = inst_names
data_named = data_still_needs_headers

In [172]:
data_named.to_csv('data_named.csv', encoding='utf-8')

In [174]:
data_z_named = stats.zscore(data_named, axis=1)
data_z_full_named = pd.DataFrame(data=data_z_named,columns=data_super_nans.columns)

In [188]:
data_z_full_named.index = data_named.index

In [189]:
data_z_full_named.ix[:,'UNITID'] = data_named.ix[:,'UNITID']

In [191]:
data_z_full_named.to_csv('data_z_full_final_named.csv', encoding='utf-8')

In [ ]:
y_full_named = data_named.ix[:,'mn_earn_wne_p6']
y_full_z_named = data_z_full_named.ix[:,'mn_earn_wne_p6']
x_full_name = data_named.drop('mn_earn_wne_p6', axis=1)
x_full_z = data_z_full_named.drop('mn_earn_wne_p6', axis=1)